In [2]:
import polars as pl

In [2]:
df = pl.read_ndjson('./experiments/base_new/log-913.ndjson', infer_schema_length=None)

In [3]:
df_schema = df.schema

In [28]:
df = pl.scan_ndjson('./experiments/base_new2/log-9*.ndjson').collect()

#df = df.filter(pl.col('symmetric') == True)
#df = df.filter(pl.col('comparison_category') == 'riod_pooled_citest')
#df = df.filter(pl.col('llf_neg_prob_fix') == 'clipping')
df = df.filter(pl.col('num_samples') >= 100)
#df = df.filter(pl.col('alpha') == 0.01)
len(df)

36000

In [29]:
_df = df
_df = _df.with_columns(accuracy=pl.col('tp') + pl.col('tn'))

_df = _df.with_columns(comparison_type=pl.col('name').str.slice(0,3))
_df = _df.with_columns(comparison_variables=pl.col('name').str.slice(0,3))
_df = _df.with_columns(comparison_type=pl.col('name').str.slice(0,3))

grouping_keys = ['comparison_type', 'num_samples', 'alpha', 'comparison_category']
by_keys = ['name', 'num_clients']
y_key = 'accuracy'

_df.plot.box(y=y_key,
             by=by_keys,
             groupby=grouping_keys,
             c=by_keys[0],
             rot=60,
             cmap='Pastel2',
             ylim=(-0.1,1.1),
             legend=False,
             widget_location='bottom'
             )

BokehModel(combine_events=True, render_bundle={'docs_json': {'13ea2600-4449-40a3-ac60-f5bbfd491828': {'version…

In [30]:
__df = df.select('name', 'num_clients', 'num_samples', 'symmetric', 'alpha', 'predicted_p_vals', 'true_p_vals') \
    .explode('predicted_p_vals', 'true_p_vals') \
    .group_by('name', 'symmetric', 'num_clients', 'num_samples', 'alpha') \
    .agg(pl.corr('predicted_p_vals', 'true_p_vals')) \
    .rename({'predicted_p_vals': 'p_value_correlation'}) 
    
__df = __df.with_columns(comparison_type=pl.col('name').str.slice(0,3))
    
__df.sort('num_samples').plot.line(x='num_samples', y='p_value_correlation',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'alpha'], ylim=(0.8,1.001))

BokehModel(combine_events=True, render_bundle={'docs_json': {'ce530e4a-1eb1-485d-844a-db75dd049854': {'version…

In [31]:
__df = df

__df = __df.with_columns(comparison_type=pl.col('name').str.slice(0,3))

#_df_decision_agreements.sort('num_samples').plot.line(x='num_samples', y='accuracy',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix'], ylim=(-0.1,1.1))


__df = __df.explode('predicted_p_vals', 'true_p_vals')
__df = __df.with_columns(decision_agreement=((pl.col('predicted_p_vals') > pl.col('alpha')) &
                                                               (pl.col('true_p_vals') > pl.col('alpha'))) |
                                                               ((pl.col('predicted_p_vals') < pl.col('alpha')) &
                                                               (pl.col('true_p_vals') < pl.col('alpha')))
                                                               )

grouping_cols = ['num_samples', 'name', 'comparison_type','num_clients', 'symmetric', 'alpha']

__df = __df.group_by(grouping_cols).agg(pl.col('decision_agreement').mean())

__df.sort('num_samples').plot.line(x='num_samples', y='decision_agreement',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'symmetric', 'alpha'], ylim=(0.95,1.01))

BokehModel(combine_events=True, render_bundle={'docs_json': {'c60f9dff-b493-423a-bc80-854fa7c6f5bb': {'version…

In [32]:
_df_decision_agreements = _df

#_df_decision_agreements.sort('num_samples').plot.line(x='num_samples', y='accuracy',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix'], ylim=(-0.1,1.1))


_df_decision_agreements = _df_decision_agreements.with_columns(accuracy=pl.col('tn')+pl.col('tp'))

grouping_cols = ['num_samples', 'name', 'comparison_type','num_clients', 'symmetric', 'alpha']

_df_decision_agreements = _df_decision_agreements.group_by(grouping_cols).agg(pl.col('accuracy').mean())

_df_decision_agreements.sort('num_samples').plot.line(x='num_samples', y='accuracy',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'symmetric', 'alpha'], ylim=(0.95,1.01))

BokehModel(combine_events=True, render_bundle={'docs_json': {'e4b1ab5c-8e62-4a9e-9249-127293d58f0f': {'version…

In [8]:
_df.explode('predicted_p_vals', 'true_p_vals').plot.scatter(x='predicted_p_vals',
                                                             y='true_p_vals',
                                                             groupby=['name', 'num_samples', 'num_clients'],
                                                             ylim=(-0.1,1.1),
                                                             xlim=(-0.1,1.1),
                                                             width=400,
                                                             height=400,
                                                             alpha=0.5
                                                             )

BokehModel(combine_events=True, render_bundle={'docs_json': {'30199118-5ae5-415e-bba7-d3021fd4592e': {'version…

In [9]:
__df = _df
__df = __df.explode('predicted_p_vals', 'true_p_vals')
__df = __df.with_columns(decision_agreement=((pl.col('predicted_p_vals') > pl.col('alpha')) &
                                            (pl.col('true_p_vals') > pl.col('alpha'))) |
                                            ((pl.col('predicted_p_vals') < pl.col('alpha')) &
                                            (pl.col('true_p_vals') < pl.col('alpha')))
                                            )


print(__df['decision_agreement'].value_counts())
print(__df.filter(~pl.col('decision_agreement')).group_by('name').len().sort('len', descending=True))
print(__df.filter(~pl.col('decision_agreement')).group_by('num_clients').len().sort('len', descending=True))
print(__df.filter(~pl.col('decision_agreement')).group_by('name', 'num_samples', 'num_clients').len().sort('len', descending=True))

__df.filter(~pl.col('decision_agreement')).plot.scatter(x='predicted_p_vals',
                y='true_p_vals',
                #by='decision_agreement',
                #by=['name', 'num_samples', 'num_clients'],
                groupby=['name', 'num_samples', 'num_clients'],#, 'llf_neg_prob_fix', 'alpha'],
                ylim=(-0.1,1.1),
                xlim=(-0.1,1.1),
                width=400,
                height=400,
                alpha=0.5
                )

shape: (2, 2)
┌────────────────────┬────────┐
│ decision_agreement ┆ count  │
│ ---                ┆ ---    │
│ bool               ┆ u32    │
╞════════════════════╪════════╡
│ false              ┆ 2168   │
│ true               ┆ 501832 │
└────────────────────┴────────┘
shape: (8, 2)
┌─────────────────┬─────┐
│ name            ┆ len │
│ ---             ┆ --- │
│ str             ┆ u32 │
╞═════════════════╪═════╡
│ L-M Con. Indep. ┆ 574 │
│ L-M Con. Dep.   ┆ 552 │
│ L-B Con. Dep.   ┆ 412 │
│ L-B Con. Indep. ┆ 359 │
│ L-M Unc. Dep.   ┆ 83  │
│ L-B Unc. Dep.   ┆ 78  │
│ L-M Unc. Indep. ┆ 70  │
│ L-B Unc. Indep. ┆ 40  │
└─────────────────┴─────┘
shape: (3, 2)
┌─────────────┬──────┐
│ num_clients ┆ len  │
│ ---         ┆ ---  │
│ i64         ┆ u32  │
╞═════════════╪══════╡
│ 5           ┆ 1160 │
│ 3           ┆ 764  │
│ 1           ┆ 244  │
└─────────────┴──────┘
shape: (203, 4)
┌─────────────────┬─────────────┬─────────────┬─────┐
│ name            ┆ num_samples ┆ num_clients ┆ len │
│ ---  

BokehModel(combine_events=True, render_bundle={'docs_json': {'a9e07e77-ebda-4cf6-8ecf-cb985a9bcc52': {'version…

In [10]:
df = pl.scan_ndjson('./experiments/base_new/log-9*.ndjson', schema=df_schema).collect()

df = df.filter(pl.col('num_samples') >= 100)
#df = df.filter(pl.col('alpha') == 0.01)

len(df)

NameError: name 'df_schema' is not defined

In [ ]:
_df = df
_df = _df.explode('predicted_p_vals', 'true_p_vals')
_df = _df.with_columns(decision_agreement=((pl.col('predicted_p_vals') > pl.col('alpha')) &
                                            (pl.col('true_p_vals') > pl.col('alpha'))) |
                                            ((pl.col('predicted_p_vals') < pl.col('alpha')) &
                                            (pl.col('true_p_vals') < pl.col('alpha')))
                                            )

_df = _df.with_columns(comparison_variables=pl.col('name').str.slice(0,3))
_df = _df.with_columns(comparison_type=pl.col('name').str.slice(4))

_df = _df.group_by(['num_samples', 'comparison_variables', 'comparison_type', 'num_clients', 'alpha']).agg(pl.col('decision_agreement').mean())

#cat_dtype = pl.Enum(('L-B', 'L-M', 'L-O', 'B-O', 'M-O'))
#_df = _df.with_columns(pl.col('comparison_variables').cast(cat_dtype))

_df = _df.sort('comparison_variables','num_samples')


_df.plot.line(x='num_samples',
                y='decision_agreement',
                alpha=0.8,
                #line_dash=dash_patterns,
                #cmap=colrs,
                #line_width=3,
                by='comparison_variables',
                groupby=['comparison_type','num_clients', 'alpha'],
                ylim=(0.95,1.001)) #* _df.plot.errorbars(y='decision_agreement', yerr1='std')

BokehModel(combine_events=True, render_bundle={'docs_json': {'a7fedc69-c50a-4e0d-b74b-f5121ade7a98': {'version…

In [ ]:
_df = df.select('name', 'num_clients', 'num_samples', 'symmetric', 'alpha', 'predicted_p_vals', 'true_p_vals') \
    .explode('predicted_p_vals', 'true_p_vals') \
    .group_by('name', 'num_clients', 'num_samples', 'alpha') \
    .agg(pl.corr('predicted_p_vals', 'true_p_vals')) \
    .rename({'predicted_p_vals': 'p_value_correlation'}) 
    
_df = _df.with_columns(comparison_variables=pl.col('name').str.slice(0,3))
_df = _df.with_columns(comparison_type=pl.col('name').str.slice(4))
    
_df.sort('num_samples').plot.line(x='num_samples',
                                  y='p_value_correlation',
                                  alpha=0.6,
                                  by='comparison_variables',
                                  groupby=['comparison_type','num_clients', 'alpha'],
                                  ylim=(0.95,1.001))

BokehModel(combine_events=True, render_bundle={'docs_json': {'54a69991-fc8f-4d3c-811d-c2df38f8662b': {'version…

In [ ]:
_df = df
_df = _df.explode('predicted_p_vals', 'true_p_vals')


_df = _df.with_columns(comparison_variables=pl.col('name').str.slice(0,3))
_df = _df.with_columns(comparison_type=pl.col('name').str.slice(4))

#_df = _df.with_columns(pl.col(''))


_df.sort('num_samples').plot.line(x='num_samples',
                                   y='decision_agreement',
                                   alpha=0.8,
                                   by='comparison_variables',
                                   groupby=['comparison_type','num_clients', 'alpha'],
                                   ylim=(-0.01,1.001))

ValueError: 'decision_agreement' is not in list

In [ ]:
# PLOTS FROM PAPER

In [ ]:
__df = df.select('name', 'num_clients', 'num_samples', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix', 'predicted_p_vals', 'true_p_vals') \
    .explode('predicted_p_vals', 'true_p_vals') \
    .group_by('name', 'symmetric', 'num_clients', 'num_samples', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix') \
    .agg(pl.corr('predicted_p_vals', 'true_p_vals')) \
    .rename({'predicted_p_vals': 'p_value_correlation'}) 
    
__df = __df.with_columns(comparison_type=pl.col('name').str.slice(0,3))
    
__df.sort('num_samples').plot.line(x='num_samples', y='p_value_correlation',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix'])

In [78]:
import glob
import polars as pl

In [80]:
# for file in glob.glob('./experiments/base/*.ndjson'):
#     #print(file)
#     df = pl.read_ndjson(file)
#     #print(len(df))
#     df = df.filter(pl.col('num_samples') >= 100)
#     #print(len(df))
#     #break
#     df = df.write_ndjson(file)